In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
import os
from os import path
import weblogo
import seqlogo

In [5]:
filteredGeneList = pd.read_csv('../../Database/filteredGenesDetails_human_240118.txt',sep='\t')

In [6]:
acc2gene={}
for idx,row in filteredGeneList.iterrows():
    acc2gene[row['AccNum']] = row['GeneName']
acc2gene['NM_017582']

'UBE2Q1'

In [7]:
def get_fa(filename,onlyKeys = 'All'):
    keyD = dict()
    if onlyKeys!='All':
        for k in onlyKeys: keyD[k] = None
        
    def addEntry(current,d):
        entry = current.split('\n')
        if entry[0]!='' and (onlyKeys=='All' or keyD.has_key(entry[0])):
            d[entry[0]] = ''.join(entry[1:])

    d = dict()
    bf = open(filename)
    current = ['\n']
    for line in bf:
        if line[0]=='>':
            addEntry(''.join(current),d)
            if len(line)>1: current = [line[1:]]
            else: current = ['\n']
        else: current.append(line)
    addEntry(''.join(current),d)
    bf.close()
    return d

In [9]:
splicedGenes = get_fa('../../Database/splicedGenes_240118.txt')
print(type(splicedGenes))
print(list(splicedGenes.keys())[0:3])

<class 'dict'>
['NR_148357_chr1(+):11868-14361', 'NR_046018_chr1(+):11873-14408', 'NR_024540_chr1(-):14361-29369']


In [10]:
def get_fa(filename,onlyKeys = 'All'):
    keyD = dict()
    if onlyKeys!='All':
        for k in onlyKeys: keyD[k] = None
        
    def addEntry(current,d):
        entry = current.split('\n')
        if entry[0]!='' and (onlyKeys=='All' or keyD.has_key(entry[0])):
            d[entry[0]] = ''.join(entry[1:])

    d = dict()
    bf = open(filename)
    current = ['\n']
    for line in bf:
        if line[0]=='>':
            addEntry(''.join(current),d)
            if len(line)>1: current = [line[1:]]
            else: current = ['\n']
        else: current.append(line)
    addEntry(''.join(current),d)
    bf.close()
    return d

In [11]:
## truncate the keys to only accNum
ks = list(splicedGenes.keys())
for key in ks:
    splicedGenes[key.split('_chr')[0]] = splicedGenes[key]
    splicedGenes.pop(key)
print(type(splicedGenes))
print(list(splicedGenes.keys())[0:3])

<class 'dict'>
['NR_148357', 'NR_046018', 'NR_024540']


In [13]:
splicedORFs = get_fa('../../Database/splicedORFs_240118.txt')
print(type(splicedORFs))
print(list(splicedORFs.keys())[0:3])

<class 'dict'>
['NM_017582', 'NM_001276267', 'NM_173469']


In [14]:
fpUTR={}
tpUTR = {}
ORFs = {}
for idx,row in filteredGeneList.iterrows():
    accNum = row['AccNum']
    fpLen = row['fpUTR_length']
    tpLen = row['tpUTR_length']
    fpUTR[accNum] = splicedGenes[accNum][0:fpLen]
    tpUTR[accNum] = splicedGenes[accNum][-tpLen:]
    ORFs[accNum] = splicedORFs[accNum]

## Generate Seqs (edit key only)

In [34]:
key='TE_t1_t0'
deseq = pd.read_csv('../../Deseq2 Analysis/shift15/coding/csv_raw/'+key+'.csv')
deseq = deseq.sort_values(by=['log2FoldChange'])
print(len(deseq))
deseq.head(2)

7990


,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
7479,NM_000118,1139.388864,-2.651024,0.448170,-5.915216,3.314412e-09,0.000001
6252,NM_002332,2459.045499,-2.583966,0.611025,-4.228902,2.348341e-05,0.001247


In [35]:
for i in range(0,5):
    seqs=[]
    left = len(deseq)//5*i
    right = len(deseq)//5*(i+1)
    d = deseq.iloc[left:right]
    filename = "orderByTE/seq/"+key+"_"+str(i*20)+"to"+str(i*20+20)+"percentile.seq"
    with open(filename,'w') as f:
        for gene in d['Unnamed: 0']:
            if len(fpUTR[gene])<8:
                continue
            fp = fpUTR[gene]
            f.write("%s\n"%(fp[len(fp)-8:]+ORFs[gene][:11]))

## Generate seqlogos
first def helper functions

In [40]:
def file2count(fname,seqLen=19):
    raw_count = np.zeros((seqLen,4))
    with open(fname) as f:
        lines = f.read().splitlines()
    lines_tr=[''.join(s) for s in zip(*lines)]
    print('#lines = ',len(lines))
    for i in range(seqLen):
        raw_count[i][0] = lines_tr[i].count('A')
        raw_count[i][1] = lines_tr[i].count('C')
        raw_count[i][2] = lines_tr[i].count('G')
        raw_count[i][3] = lines_tr[i].count('T')
    return raw_count

In [41]:
def plotlogo(datset,raw_count):
    pfm_pd=pd.DataFrame(raw_count)
    ppm_pd=seqlogo.pfm2ppm(pfm_pd)
    ppm = seqlogo.Ppm(ppm_pd)
    seqlogo.seqlogo(ppm, ic_scale = True, format = 'png', size = 'xlarge',filename=datset+'.png')
    seqlogo.seqlogo(ppm, ic_scale = True, format = 'svg', size = 'medium',filename=datset+'.svg')

In [42]:
len(seq)

19

In [50]:
key='TE_t0'
for i in range(0,5):
    filename = "orderByTE/seq/"+key+"_"+str(i*20)+"to"+str(i*20+20)+"percentile.seq"
    raw_count = file2count(filename)
    dataset = "orderByTE/png/"+key+"_"+str(i*20)+"to"+str(i*20+20)+"percentile"
    plotlogo(dataset,raw_count)

#lines =  1565
#lines =  1568
#lines =  1587
#lines =  1581
#lines =  1589
